In [54]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle

In [36]:
def get_clean_data():
    data = pd.read_csv("data/data.csv")
    data = data.drop(['Unnamed: 32', 'id'], axis=1)
    data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0})

    return data

Đọc tệp data.csv: Đây là bước cần thiết vì nó tải dữ liệu gốc mà ta sẽ làm việc lên môi trường lập trình. Tệp này chứa thông tin về các trường hợp ung thư vú.

- Loại bỏ các cột 'Unnamed: 32' và 'id':

    - Cột 'Unnamed: 32' thường xuất hiện khi có một cột trống không có tiêu đề trong tệp CSV, nó không chứa thông tin hữu ích cho phân tích hoặc mô hình.
    - Cột 'id' chứa một định danh duy nhất cho mỗi bản ghi. Mặc dù quan trọng để xác định các trường hợp riêng lẻ, nhưng nó không phải là một đặc trưng (feature) giúp dự đoán ung thư. Do đó, việc loại bỏ nó giúp mô hình không bị nhầm lẫn và giảm nhiễu.

- Ánh xạ cột 'diagnosis' từ {'M': 1, 'B': 0}:

    - Cột 'diagnosis' ban đầu chứa các giá trị chuỗi là 'M' (Malignant - Ác tính) và 'B' (Benign - Lành tính). Hầu hết các thuật toán học máy đều hoạt động với dữ liệu số.
    - Việc chuyển đổi 'M' thành 1 và 'B' thành 0 giúp biểu diễn các loại chẩn đoán này dưới dạng số mà mô hình có thể dễ dàng xử lý. Đây là một bước chuẩn bị dữ liệu quan trọng để huấn luyện mô hình phân loại.

In [43]:
def create_model(data):
    X = data.drop(['diagnosis'], axis=1)
    y = data['diagnosis']

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #train
    model = LogisticRegression()
    model.fit(X_train, y_train)

    return  model, scaler, X_test, y_test


1. Tách Đặc trưng và Biến Mục tiêu: Đầu tiên, nó tách dữ liệu đầu vào (data) thành các đặc trưng (X), là tất cả các cột trừ 'diagnosis', và biến mục tiêu (y), chính là cột 'diagnosis'.

2. Chuẩn hóa Đặc trưng: Nó khởi tạo một StandardScaler và áp dụng nó cho các đặc trưng (X). Bước này chuẩn hóa các đặc trưng bằng cách loại bỏ giá trị trung bình và chia tỷ lệ để có phương sai đơn vị. Điều này rất quan trọng đối với nhiều thuật toán học máy, bao gồm cả Hồi quy Logistic, để hoạt động tối ưu.

3. Chia tập huấn luyện và kiểm tra: Sau đó, nó chia các đặc trưng đã được chuẩn hóa (X) và biến mục tiêu (y) thành các tập huấn luyện và kiểm tra bằng cách sử dụng train_test_split. 80% dữ liệu được sử dụng để huấn luyện (X_train, y_train) và 20% để kiểm tra (X_test, y_test), với random_state=42 đảm bảo khả năng tái tạo của việc chia tách.

4. Huấn luyện Mô hình: Một mô hình LogisticRegression được khởi tạo và sau đó được huấn luyện (fit) bằng cách sử dụng dữ liệu huấn luyện (X_train, y_train). Đây là nơi mô hình học mối quan hệ giữa các đặc trưng và chẩn đoán.

5. Trả về Giá trị: Cuối cùng, hàm trả về model đã được huấn luyện và đối tượng scaler. Đối tượng scaler được trả về vì nó sẽ cần thiết để biến đổi dữ liệu mới, chưa từng thấy theo cùng một cách mà dữ liệu huấn luyện đã được biến đổi trước khi đưa ra dự đoán.

In [47]:
def test_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    print("Classification Report:\n", classification_report(y_test, y_pred))


`test_model`, chịu trách nhiệm đánh giá hiệu suất của mô hình học máy đã được huấn luyện.

Dưới đây là chi tiết về những gì nó thực hiện:

1. Dự đoán: y_pred = model.predict(X_test): Dòng này sử dụng model đã được huấn luyện để đưa ra các dự đoán (y_pred) trên các đặc trưng kiểm tra chưa từng thấy (X_test). Những dự đoán này là phỏng đoán tốt nhất của mô hình về 'chẩn đoán' dựa trên các đặc trưng đầu vào.
2. Tính độ chính xác: accuracy = accuracy_score(y_test, y_pred): Điều này tính toán độ chính xác tổng thể của mô hình bằng cách so sánh các nhãn thực tế (y_test) với các dự đoán của mô hình (y_pred). Độ chính xác là tỷ lệ các trường hợp được phân loại đúng.
3. In độ chính xác: print("Accuracy:", accuracy): Điều này hiển thị độ chính xác đã tính toán ra console.
4. Tạo Báo cáo Phân loại: print("Classification Report:\n", classification_report(y_test, y_pred)): Điều này tạo và in một báo cáo phân loại chi tiết. Báo cáo này cung cấp các chỉ số sắc thái hơn ngoài độ chính xác, chẳng hạn như độ chính xác (precision), độ thu hồi (recall) và điểm f1-score cho mỗi lớp (Ác tính và Lành tính trong trường hợp này), cùng với số lượng hỗ trợ (số lần xuất hiện thực tế của mỗi lớp trong tập kiểm tra). Điều này giúp hiểu hiệu suất của mô hình cho từng danh mục.

In [63]:
def dump_model(model,scaler):
    with open('model/model.pkl', 'wb') as f:
        pickle.dump(model, f)
    with open('model/scaler.pkl', 'wb') as f:
        pickle.dump(scaler, f)

`dump_model` nhận đầu vào là model (mô hình học máy đã được huấn luyện) và đối tượng scaler. Mục đích của nó là lưu hai đối tượng này vào các tệp bằng cách sử dụng module pickle của Python.

- Nó mở một tệp có tên 'model/model.pkl' ở chế độ ghi nhị phân ('wb') và sử dụng pickle.dump() để lưu đối tượng model vào tệp này.
- Tương tự, nó mở một tệp khác có tên 'model/scaler.pkl' ở chế độ ghi nhị phân ('wb') và sử dụng pickle.dump() để lưu đối tượng scaler vào tệp này.

Quá trình này rất cần thiết cho việc duy trì: một khi mô hình của bạn đã được huấn luyện và bộ chia tỷ lệ dữ liệu của bạn đã được khớp, bạn có thể lưu chúng lại. Điều này cho phép bạn tải chúng sau này để đưa ra dự đoán trên dữ liệu mới mà không cần phải huấn luyện lại mô hình hoặc khớp lại bộ chia tỷ lệ mỗi lần.

In [62]:
def main():
    data = get_clean_data()
    model, scaler, X_test, y_test = create_model(data)
    test_model(model, X_test, y_test)

    dump_model(model, scaler)
main()

Accuracy: 0.9736842105263158
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98        71
           1       0.98      0.95      0.96        43

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114

